# VR/AR Interaction Study — Analysis Notebook

This notebook computes descriptive stats, **paired t-tests**, **effect sizes** (Cohen's d), and summarizes **SUS** and **NASA‑TLX** for **gaze vs. joystick** conditions.

> Fill in or replace `data_template.csv` with your actual data and re-run.

In [ ]:
# Setup
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Try to import scipy; if unavailable, fall back to manual t-tests
try:
    from scipy import stats
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False

data_path = Path('/mnt/data/hci_vr_ar_project/data_template.csv')
df = pd.read_csv(data_path)
print(df.head())

# Basic cleaning
df['condition'] = df['condition'].str.lower().str.strip()
df['hit'] = df['hit'].astype(int)
df['selection_time_ms'] = pd.to_numeric(df['selection_time_ms'], errors='coerce')
df['errors'] = pd.to_numeric(df['errors'], errors='coerce')


In [ ]:
# Aggregate per participant & condition
agg = df.groupby(['participant_id','condition']).agg(
    mean_time_ms=('selection_time_ms','mean'),
    median_time_ms=('selection_time_ms','median'),
    error_rate=('errors', lambda x: np.sum(x)/max(len(x),1))
).reset_index()

# Wide format for paired tests
wide_time = agg.pivot(index='participant_id', columns='condition', values='mean_time_ms')
wide_error = agg.pivot(index='participant_id', columns='condition', values='error_rate')

display(agg.head())
print("\nWide (time):\n", wide_time.head())
print("\nWide (error):\n", wide_error.head())


In [ ]:
# Helper: paired t-test with effect size
def paired_test(x, y, label):
    x = pd.to_numeric(x, errors='coerce').dropna()
    y = pd.to_numeric(y, errors='coerce').dropna()
    common = x.index.intersection(y.index)
    x = x.loc[common]
    y = y.loc[common]
    diff = x - y
    n = len(diff)
    if n < 2:
        print(f'Not enough data for paired test on {label}.')
        return None
    if SCIPY_OK:
        t, p = stats.ttest_rel(x, y, nan_policy='omit')
    else:
        # manual paired t-test
        mean_diff = diff.mean()
        sd_diff = diff.std(ddof=1)
        t = mean_diff / (sd_diff / np.sqrt(n))
        # two-tailed p-value via normal approx (fallback)
        from math import erf, sqrt
        p = 2 * (1 - 0.5*(1+erf(abs(t)/sqrt(2))))
    # Cohen's d for paired samples (dz)
    dz = diff.mean() / diff.std(ddof=1)
    print(f"{label}: n={n}, t={t:.3f}, p={p:.4f}, Cohen's dz={dz:.3f}")
    return {'n': n, 't': t, 'p': p, 'dz': dz}

res_time = paired_test(wide_time.get('gaze'), wide_time.get('joystick'), 'Mean time (ms): gaze vs joystick')
res_error = paired_test(wide_error.get('gaze'), wide_error.get('joystick'), 'Error rate: gaze vs joystick')


In [ ]:
# SUS scoring (0-100). We expect columns sus_q1..sus_q10 per row.
def compute_sus(row):
    pos = [1,3,5,7,9]
    neg = [2,4,6,8,10]
    score = 0
    for i in pos:
        score += (row[f'sus_q{i}'] - 1)
    for i in neg:
        score += (5 - row[f'sus_q{i}'])
    return score * 2.5

# Compute per participant per condition SUS (if present)
if all(col in df.columns for col in [f'sus_q{i}' for i in range(1,11)]):
    sus = df.groupby(['participant_id','condition']).apply(lambda g: compute_sus(g.iloc[0])).reset_index(name='sus_score')
    wide_sus = sus.pivot(index='participant_id', columns='condition', values='sus_score')
    res_sus = None
    if 'gaze' in wide_sus and 'joystick' in wide_sus:
        res_sus = (wide_sus['gaze'] - wide_sus['joystick']).describe()
        print("\nSUS summary (gaze - joystick):\n", res_sus)
else:
    print("SUS columns not found; skipping SUS scoring.")


In [ ]:
# NASA-TLX summary (0-100 per subscale)
nasatlx_cols = ['nasatlx_mental','nasatlx_physical','nasatlx_temporal','nasatlx_performance','nasatlx_effort','nasatlx_frustration']
if all(c in df.columns for c in nasatlx_cols):
    ntlx = df.groupby(['participant_id','condition'])[nasatlx_cols].mean().reset_index()
    display(ntlx.head())
else:
    print("NASA-TLX columns not found; skipping NASATLX summary.")


In [ ]:
# Plots (Matplotlib, one chart per figure, default colors)
# Time by condition
cond_time = agg.groupby('condition')['mean_time_ms'].mean().reset_index()
plt.figure()
plt.bar(cond_time['condition'], cond_time['mean_time_ms'])
plt.title('Mean Selection Time by Condition')
plt.xlabel('Condition')
plt.ylabel('Time (ms)')
plt.show()

# Error rate by condition
cond_error = agg.groupby('condition')['error_rate'].mean().reset_index()
plt.figure()
plt.bar(cond_error['condition'], cond_error['error_rate'])
plt.title('Mean Error Rate by Condition')
plt.xlabel('Condition')
plt.ylabel('Error Rate')
plt.show()
